In [1]:
import numpy as np

In [294]:
interval = [-5.12, 5.12]
n = 10
l = 10
# interval = [2, 5]

In [295]:
def rastrigin(X, A = 10):
    
    """ Rastrigin function. """
    
    n = len(X)
    Y = A*n + sum([np.square(x) - A*np.cos(2*np.pi*x) for x in X])
    
    return Y

In [296]:
def decoding(B, l, interval):
    
    """ Decoding each binary solution to a floating point representation. """
    
    n = int(len(B)/l)
    X = np.zeros((n,))
    
    # The interval between adjacent values
    delta = (interval[1] - interval[0])/(np.power(2,l)-1)  
    
    # For each variable b in B
    for i in range(n):
        b = B[l*i:l*(i+1)]
        X[i] = interval[0] + delta * sum([b[k]*np.power(2,(l-k-1)) for k in range(l)])

    return X

In [305]:
def encoding(X, l, interval):
    
    """ Encoding each real solution to a binary representation. """
    
    n = len(X)
    B = list()
    
    # For each variable X[i]
    for i in range(n):        
        k = int(np.ceil((X[i] - interval[0])*(np.power(2,l)-1)/(interval[1] - interval[0])))
        b = bin(k).replace("0b", "")
        b = [float(bi) for bi in b]
        
        if len(b) < l:
            b = list(np.zeros((l-len(b),))) + b
            
        B.append(b)    
    
    B = np.array(B).reshape((-1))
    
    return B

In [318]:
X = np.zeros((n,))
#X = np.array([-5.12]*l)
print(X)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [319]:
B = encoding(X, l = l, interval = interval)
B

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [320]:
X = decoding(B, l = l, interval = interval)
X

array([0.00500489, 0.00500489, 0.00500489, 0.00500489, 0.00500489,
       0.00500489, 0.00500489, 0.00500489, 0.00500489, 0.00500489])

In [321]:
rastrigin(X)

0.04969096075727464